In [4]:
import time
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import resnet50, efficientnet, mobilenet
from PIL.ImageDraw import Draw
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator

In [5]:
META_FILE = 'datasets/labels.csv'
data = pd.read_csv(META_FILE)

In [6]:
def reconstruct_path(image_id):
    """Превращает номерной ID изображения в относительный путь.
    """
    image_id = str(image_id).rjust(6, '0')
    return f'datasets/frames/seq_{image_id}.jpg'


data['path'] = data['id'].apply(reconstruct_path)
data.head()

,id,count,path
0,1,35,datasets/frames/seq_000001.jpg
1,2,41,datasets/frames/seq_000002.jpg
2,3,41,datasets/frames/seq_000003.jpg
3,4,44,datasets/frames/seq_000004.jpg
4,5,41,datasets/frames/seq_000005.jpg


In [7]:
def load_image(is_labelled, is_training=True):
    """Загружает либо просто изображение, если ему не соответствует значение, либо изображение вместе со значением.
    """
    def _get_image(path):
        """Загружает изображение из файла и меняет его случайным образом.
        """
        image = tf.image.decode_jpeg(tf.io.read_file(path), channels=3)
        image = tf.cast(image, dtype=tf.int32)
        image = tf.image.resize_with_pad(image, 299, 299)
        if is_training:
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_brightness(image, 0.1)
            image = tf.image.random_contrast(image, 0.1, 0.2)
            image = tf.image.random_saturation(image, 0.9, 1.1)
            image = tf.image.random_hue(image, 0.1)
        return tf.keras.applications.inception_resnet_v2.preprocess_input(image)

    def _get_image_label(img, label):
        """Загружает изображение вместе с соответствующим ему значением.
        """
        return _get_image(img), label

    return _get_image_label if is_labelled else _get_image


def prepare_dataset(dataset, is_training=True, is_labeled=True):
    """Меняет датасет так, чтобы он содержал тензоры изображений и соответствующие им значения.
    """
    image_read_fn = load_image(is_labeled, is_training)
    dataset = dataset.map(image_read_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)


def create_model():
    """Инициализирует модель, меняет её верхние слои, задаёт необходимые параметры.
    """
    feature_model = tf.keras.applications.InceptionResNetV2(
        include_top=False, pooling='avg')
    feature_model.trainable = False

    model = tf.keras.Sequential([
        tf.keras.Input((299, 299, 3)),
        feature_model,
        Dense(512, activation='selu'),
        Dense(1)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=[tf.keras.metrics.MeanAbsoluteError()])

    return model


def plot_history(hist):
    """Создаёт график с оценкой обучения и ошибок.
    """
    mae = hist.history['mean_absolute_error']
    val_mae = hist.history['val_mean_absolute_error']
    x_axis = range(1, len(mae) + 1)
    plt.plot(x_axis, mae, 'bo', label='Training')
    plt.plot(x_axis, val_mae, 'ro', label='Validation')
    plt.title('MAE')
    plt.legend()
    plt.xlabel('Epochs')
    plt.tight_layout()
    plt.show()

In [8]:
data_train = data.head(1700) #для обучения
data_valid = data.tail(300) #для проверки

ds_train = tf.data.Dataset.from_tensor_slices((data_train['path'], data_train['count']))
ds_valid = tf.data.Dataset.from_tensor_slices((data_valid['path'], data_valid['count']))

ds_train = prepare_dataset(ds_train)
ds_valid = prepare_dataset(ds_valid, is_training=False) #не для обучения, поэтому изображения изменяться не будут

# Часть 3: ResNet50

**Модель:** CNN (Свёрточная нейронная сеть) с 50 слоями, загружается из Keras. Включает в себя возможность изменить входной и выходные слои.

#### Алгоритм
- Преобразовать изображения под формат входного слоя модели (сделаем формат изображений 224х224, минимальный для сетей ResNet).
- Загрузить модель, просмотреть её структуру и изменить верхние слои для нашей задачи.
- Обучить модель на данных датасета (при этом заморозив нижние слои) и определить, насколько хорошо она справляется с задачей распознавания человека в толпе.

Загрузим модель и зададим ей необходимые параметры.

In [9]:
resnet = resnet50.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3),
    pooling='avg',
)

In [10]:
x = resnet.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='linear')(x)
rn_model = tf.keras.Model(inputs=resnet.input, outputs=predictions)

In [11]:
rn_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
    loss="mean_squared_error",
    metrics=['mean_absolute_error', 'mean_squared_error',]
)

In [12]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    preprocessing_function=resnet50.preprocess_input,
)

In [13]:
flow_params = dict(
    dataframe=data,
    x_col='path',
    y_col='count',
    weight_col=None,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=4,
    shuffle=True,
    seed=0,
)

train_generator = datagen.flow_from_dataframe(
    subset='training',
    **flow_params    
)
valid_generator = datagen.flow_from_dataframe(
    subset='validation',
    **flow_params
)

Found 1600 validated image filenames.
Found 400 validated image filenames.


In [14]:
for layer in rn_model.layers[:-8]:
    layer.trainable = False
for layer in rn_model.layers[-8:]:
    layer.trainable = True

In [15]:
tf.debugging.set_log_device_placement(True)

In [16]:
start = time.process_time()
history = rn_model.fit(train_generator, epochs=10, validation_data=valid_generator, verbose=2, ) 
end = time.process_time()
rn_time = round(end - start, 2)

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
plot_history(history)

In [ ]:
mae_rn = history.history['mean_absolute_error'][49]
mse_rn = history.history['mean_squared_error'][49]
print(f'Validation MSE = {mse_rn}\n'
      f'Validation MAE = {mae_rn}')

In [ ]:
rn_model

In [ ]:
# rn_model.save('/kaggle/working/model_crowd_count.h5')